
#  <center><font color='FC4C02' size="5">*RECON - Cash Management*</font></center>





#### <font color='FC4C02'>Situación </font>
Santander tiene una relacion como banco adquiriente con Clip, esto quiere decir que los pagos realizados con clip en principio los adquiere Santander y luego de manera diaria Santander deposita todos estos pagos a clip, la conciliación busca validar que toda transacción realizada en clip que este en su base de datos sea depositada por Santander.

Clip y Santander consideran una partida como conciliable toda aquella transacción cuyo  estado en la base de datos  sea  <font color='FC4C02'>**PAGADA**</font> y que no este duplicada factores externos.

Para esto, <font color='FC4C02'>**Clip**</font> comparará para cada transacción campos únicos entre las dos entidades buscando encontrar parejas que sean exactamente iguales bajo las siguientes condiciones:

* Que tengan el mismo <font color='FC4C02'>ID</font>.
* Que tenga los mismos <font color='FC4C02'>6 primeros</font> dígitos de la tarjeta.
* Que tengan los mismos <font color='FC4C02'>4 últimos</font> dígitos de la tarjeta.
* Que el <font color='FC4C02'>valor pagado</font> en la transacción sea igual o que su diferencia esté en el rango de más o menos 0.99 pesos.
* Que tengan la misma <font color='FC4C02'>fecha de transacción</font>.



In [1]:
import requests
import subprocess
import os

def file_download(filename,url):
  if os.path.exists(filename):
    print("El archivo ya existe.")
  else:
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print('Archivo descargado exitosamente.')
    else:
        print('Error al descargar el archivo. Código de estado:', response.status_code)


def install_spark():
  command = "pip install pyspark"
  result = subprocess.run(command, shell=True, capture_output=True, text=True)
  if result.returncode == 0:
      print("Salida del comando:")
      print(result.stdout)
  else:
      print("Error:", result.stderr)

url_clip = 'https://firebasestorage.googleapis.com/v0/b/prueba-tecnica-43c45.appspot.com/o/clip_v2.csv?alt=media&token=b62257f8-8382-4876-9d78-d52ef6925dea'
url_santander = 'https://firebasestorage.googleapis.com/v0/b/prueba-tecnica-43c45.appspot.com/o/SANTANDER_DB.csv?alt=media&token=4fd34436-cc95-41b5-ac39-faad0c5fef39'

file_download('CLIP.csv',url_clip)
file_download('SANTANDER.csv',url_santander)
install_spark()

El archivo ya existe.
El archivo ya existe.
Salida del comando:



In [2]:
## TODO
import pyspark
from pyspark.sql import SparkSession, functions, window
spark = SparkSession.builder.appName('testclip').getOrCreate()
df_clip=spark.read.csv('CLIP.csv', header = True)
df_santander=spark.read.csv('SANTANDER.csv', header = True)
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED") # opcional si se requiere en la transformación de fechas
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")  # opcional si se requiere en la transformación de fechas

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

In [4]:
df_clip.show()

+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|INICIO6_TARJETA|FINAL4_TARJETA| TIPO_TRX|   MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|       ID_BANCO|FECHA_RECEPCION_BANCO|
+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|         443715|           190|CANCELADA|21944.00|       2023-11-01|             136719| 47-100-185-686|           2023-11-05|
|         802414|          6615|   PAGADA|   42.51|       2023-11-01|              53042| 29-607-730-901|           2023-11-02|
|         818156|           672|NO APLICA|   20.80|       2023-11-01|               null|497-363-207-741|           2023-11-02|
|         954015|          5581|   PAGADA|   41.00|       2023-11-01|              13616|441-622-557-487|           2023-11-02|
|         954015|          2538|   PAGADA|   68.00|       2023-11-01|              13602|266-554-946-896

In [5]:
df_santander.show()

+------+------+--------+------+-----------------+-------------------+---------------+---------------+
|   bin|last_4|TIPO_TRX| monto|FECHA_TRANSACCION|codigo_autorizacion| ID_ADQUIRIENTE|FECHA_RECEPCION|
+------+------+--------+------+-----------------+-------------------+---------------+---------------+
|954015|  0300|    PAGO|    44|       2023-11-01|             272012|320-720-237-226|     2023-03-11|
|954015|  0300|    PAGO|    93|       2023-11-01|             216149|812-801-092-295|     2023-03-11|
|954015|  0310|    PAGO|  1579|       2023-11-01|             898544|563-080-775-636|     2023-03-11|
|954015|  0319|    PAGO|    86|       2023-11-01|             501928|924-685-088-703|     2023-03-11|
|954015|  0314|    PAGO|   679|       2023-11-01|             170207| 35-452-614-774|     2023-03-11|
|954015|  0347|    PAGO|    75|       2023-11-01|               9187|603-396-947-906|     2023-03-11|
|954015|  0383|    PAGO|   525|       2023-11-01|              55165|    153-266-9

In [6]:
#Conteo inicial de registros
print(df_clip.count())
print(df_santander.count())

163549
132398


#Calidad de datos
## Valores nulos

In [7]:
def count_nulls(df):
    return df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

print("Nulos en df_clip:")
count_nulls(df_clip).show()

print("Nulos en df_santander:")
count_nulls(df_santander).show()

Nulos en df_clip:
+---------------+--------------+--------+-----+-----------------+-------------------+--------+---------------------+
|INICIO6_TARJETA|FINAL4_TARJETA|TIPO_TRX|MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|ID_BANCO|FECHA_RECEPCION_BANCO|
+---------------+--------------+--------+-----+-----------------+-------------------+--------+---------------------+
|              0|             0|       0|    0|                0|                  0|       0|                    0|
+---------------+--------------+--------+-----+-----------------+-------------------+--------+---------------------+

Nulos en df_santander:
+---+------+--------+-----+-----------------+-------------------+--------------+---------------+
|bin|last_4|TIPO_TRX|monto|FECHA_TRANSACCION|codigo_autorizacion|ID_ADQUIRIENTE|FECHA_RECEPCION|
+---+------+--------+-----+-----------------+-------------------+--------------+---------------+
|  0|     0|       0|    0|                0|                  0|             0|  

###Identificación de valores nulos como strings en campos clave
Aunque hay una precencia considerable de datos nulos como strings, por el momento no se hará un tratamiento ya que no obtaculiza las actividades posteriores de la conciliación, pues nos enfocaremos en  transacciones cuyo estado  sea **PAGADA**

In [8]:
# Filter the DataFrame and assign it to df_clip_null
df_clip_null = df_clip.filter(
    col("ID_BANCO").isin("null") | col("INICIO6_TARJETA").isin("null") | col("FINAL4_TARJETA").isin("null")
)


df_clip_null.show(50)


+---------------+--------------+---------+------+-----------------+-------------------+--------+---------------------+
|INICIO6_TARJETA|FINAL4_TARJETA| TIPO_TRX| MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|ID_BANCO|FECHA_RECEPCION_BANCO|
+---------------+--------------+---------+------+-----------------+-------------------+--------+---------------------+
|           null|          9630|NO APLICA|500.00|       2023-11-01|               null|    null|           2023-11-02|
|           null|          9630|NO APLICA|500.00|       2023-11-01|               null|    null|           2023-11-02|
|         818156|          9664|NO APLICA| 66.00|       2023-11-01|               null|    null|           2023-11-02|
|           null|          1332|NO APLICA| 12.12|       2023-11-01|               null|    null|           2023-11-02|
|           null|          1332|NO APLICA| 12.12|       2023-11-01|               null|    null|           2023-11-02|
|         975488|          7134|NO APLICA| 32.50

In [12]:

# Conteo
print(df_clip_null.count())

16


In [10]:
df_santander.filter(
    col("bin").isin("null") | col("last_4").isin("null") |col("ID_ADQUIRIENTE").isin("null")
).show()

+---+------+--------+-----+-----------------+-------------------+--------------+---------------+
|bin|last_4|TIPO_TRX|monto|FECHA_TRANSACCION|codigo_autorizacion|ID_ADQUIRIENTE|FECHA_RECEPCION|
+---+------+--------+-----+-----------------+-------------------+--------------+---------------+
+---+------+--------+-----+-----------------+-------------------+--------------+---------------+



In [11]:
#Ver tipos de datos
df_clip.printSchema()
df_santander.printSchema()

root
 |-- INICIO6_TARJETA: string (nullable = true)
 |-- FINAL4_TARJETA: string (nullable = true)
 |-- TIPO_TRX: string (nullable = true)
 |-- MONTO: string (nullable = true)
 |-- FECHA_TRANSACCION: string (nullable = true)
 |-- CODIGO_AUTORIZACION: string (nullable = true)
 |-- ID_BANCO: string (nullable = true)
 |-- FECHA_RECEPCION_BANCO: string (nullable = true)

root
 |-- bin: string (nullable = true)
 |-- last_4: string (nullable = true)
 |-- TIPO_TRX: string (nullable = true)
 |-- monto: string (nullable = true)
 |-- FECHA_TRANSACCION: string (nullable = true)
 |-- codigo_autorizacion: string (nullable = true)
 |-- ID_ADQUIRIENTE: string (nullable = true)
 |-- FECHA_RECEPCION: string (nullable = true)



In [13]:
df_santander = (
    spark.read.option("header", True).csv("SANTANDER.csv")
        .withColumnRenamed("bin",           "BIN_6")
        .withColumnRenamed("last_4",        "LAST4")
        .withColumnRenamed("TIPO_TRX",      "TIPO_TRX")
        .withColumnRenamed("monto",         "MONTO")
        .withColumnRenamed("FECHA_TRANSACCION", "FECHA_TRANSACCION")
        .withColumnRenamed("codigo_autorizacion", "CODIGO_AUTORIZACION")
        .withColumnRenamed("ID_ADQUIRIENTE", "ID_BANCO")
        .withColumnRenamed("FECHA_RECEPCION", "FECHA_RECEPCION_BANCO")
        .withColumn("MONTO",  col("MONTO").cast("double"))
        .withColumn("FECHA_TRANSACCION", to_date("FECHA_TRANSACCION"))
)


(
    df_clip.write
    .mode("overwrite")
    .option("header", True)
    .csv("output/clip_limpio")
)

(
    df_santander.write
    .mode("overwrite")
    .option("header", True)
    .csv("output/santander_limpio")
)

print("Columnas CLIP  ➜", df_clip.columns)
print("Columnas SANT. ➜", df_santander.columns)

Columnas CLIP  ➜ ['INICIO6_TARJETA', 'FINAL4_TARJETA', 'TIPO_TRX', 'MONTO', 'FECHA_TRANSACCION', 'CODIGO_AUTORIZACION', 'ID_BANCO', 'FECHA_RECEPCION_BANCO']
Columnas SANT. ➜ ['BIN_6', 'LAST4', 'TIPO_TRX', 'MONTO', 'FECHA_TRANSACCION', 'CODIGO_AUTORIZACION', 'ID_BANCO', 'FECHA_RECEPCION_BANCO']


In [14]:
df_santander.show()

+------+-----+--------+------+-----------------+-------------------+---------------+---------------------+
| BIN_6|LAST4|TIPO_TRX| MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|       ID_BANCO|FECHA_RECEPCION_BANCO|
+------+-----+--------+------+-----------------+-------------------+---------------+---------------------+
|954015| 0300|    PAGO|  44.0|       2023-11-01|             272012|320-720-237-226|           2023-03-11|
|954015| 0300|    PAGO|  93.0|       2023-11-01|             216149|812-801-092-295|           2023-03-11|
|954015| 0310|    PAGO|1579.0|       2023-11-01|             898544|563-080-775-636|           2023-03-11|
|954015| 0319|    PAGO|  86.0|       2023-11-01|             501928|924-685-088-703|           2023-03-11|
|954015| 0314|    PAGO| 679.0|       2023-11-01|             170207| 35-452-614-774|           2023-03-11|
|954015| 0347|    PAGO|  75.0|       2023-11-01|               9187|603-396-947-906|           2023-03-11|
|954015| 0383|    PAGO| 525.0|       

In [15]:
df_clip.show()

+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|INICIO6_TARJETA|FINAL4_TARJETA| TIPO_TRX|   MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|       ID_BANCO|FECHA_RECEPCION_BANCO|
+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|         443715|           190|CANCELADA|21944.00|       2023-11-01|             136719| 47-100-185-686|           2023-11-05|
|         802414|          6615|   PAGADA|   42.51|       2023-11-01|              53042| 29-607-730-901|           2023-11-02|
|         818156|           672|NO APLICA|   20.80|       2023-11-01|               null|497-363-207-741|           2023-11-02|
|         954015|          5581|   PAGADA|   41.00|       2023-11-01|              13616|441-622-557-487|           2023-11-02|
|         954015|          2538|   PAGADA|   68.00|       2023-11-01|              13602|266-554-946-896

##Tratamiento de datos  duplicados

In [16]:
key_cols_clip = ["ID_BANCO", "INICIO6_TARJETA", "FINAL4_TARJETA", "FECHA_TRANSACCION", "MONTO"]
key_cols_santander = ["ID_BANCO", "BIN_6", "LAST4", "FECHA_TRANSACCION", "MONTO"]

w_dup_b = Window.partitionBy(*key_cols_clip).orderBy(col('FECHA_TRANSACCION'))
clip_unique = df_clip.withColumn('rn', row_number().over(w_dup_b)).filter('rn = 1').drop('rn')
clip_duplicates = df_clip.withColumn('rn', row_number().over(w_dup_b)).filter('rn > 1').drop('rn')


w_dup = Window.partitionBy(*key_cols_santander).orderBy(col('FECHA_TRANSACCION'))

santander_unique = df_santander.withColumn('rn', row_number().over(w_dup)).filter('rn = 1').drop('rn')

santander_duplicates = df_santander.withColumn('rn', row_number().over(w_dup)).filter('rn > 1').drop('rn')

print("Duplicados en df_clip:")
clip_duplicates.show()

print("Duplicados en df_santander:")
santander_duplicates.show()

Duplicados en df_clip:
+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|INICIO6_TARJETA|FINAL4_TARJETA| TIPO_TRX|   MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|       ID_BANCO|FECHA_RECEPCION_BANCO|
+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|         975199|          2574|NO APLICA|  205.00|       2023-11-01|             916499|142-794-834-689|           2023-11-01|
|         443715|           190|   PAGADA|21944.00|       2023-11-01|             136719| 47-100-185-686|           2023-11-02|
|         944439|          3027|   PAGADA| 7300.00|       2023-11-01|             452762|550-902-856-935|           2023-11-01|
|         928869|          8072|CANCELADA|   20.00|       2023-11-01|               8677|632-954-196-335|           2023-11-03|
|         442732|          4189|NO APLICA| 1390.00|       2023-11-01|            

In [17]:
#Conteo de datos después de tratamiento
print(santander_unique.count())
print(clip_unique.count())

132396
163537


#Filtro de transacciones pagadas


In [18]:
df_clip_pago = clip_unique.filter(col("TIPO_TRX") == "PAGADA")

# Filtrar df_santander donde TIPO_TRX = 'PAGO'
df_santander_pago = santander_unique.filter(col("TIPO_TRX") == "PAGO")

# Para ver un muestreo
df_clip_pago.show()
df_santander_pago.show()

+---------------+--------------+--------+-------+-----------------+-------------------+-------------+---------------------+
|INICIO6_TARJETA|FINAL4_TARJETA|TIPO_TRX|  MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|     ID_BANCO|FECHA_RECEPCION_BANCO|
+---------------+--------------+--------+-------+-----------------+-------------------+-------------+---------------------+
|         452027|          8225|  PAGADA| 297.00|       2023-11-01|              85385|1-000-719-344|           2023-11-01|
|         930288|          8784|  PAGADA| 609.00|       2023-11-01|              60503|1-006-477-580|           2023-11-01|
|         975488|          8840|  PAGADA| 580.00|       2023-11-01|               1178|1-006-920-138|           2023-11-02|
|         984606|          2643|  PAGADA| 416.00|       2023-11-01|             918235|1-048-327-016|           2023-11-02|
|         449289|          4444|  PAGADA|1000.00|       2023-11-01|               1753|1-068-687-103|           2023-11-02|
|       

In [19]:
#Contar numero de filas
print(df_clip_pago.count())
print(df_santander_pago.count())

147328
132338


#Filtro de transacciones que no conforman pagos

In [20]:

df_clip_no_pago = df_clip.filter(col("TIPO_TRX") != "PAGADA")


df_santander_no_pago = df_santander.filter(col("TIPO_TRX") != "PAGO")


df_clip_no_pago.show()
df_santander_no_pago.show()

+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|INICIO6_TARJETA|FINAL4_TARJETA| TIPO_TRX|   MONTO|FECHA_TRANSACCION|CODIGO_AUTORIZACION|       ID_BANCO|FECHA_RECEPCION_BANCO|
+---------------+--------------+---------+--------+-----------------+-------------------+---------------+---------------------+
|         443715|           190|CANCELADA|21944.00|       2023-11-01|             136719| 47-100-185-686|           2023-11-05|
|         818156|           672|NO APLICA|   20.80|       2023-11-01|               null|497-363-207-741|           2023-11-02|
|         946891|          1670|NO APLICA|  776.60|       2023-11-01|             H37254|249-718-452-281|           2023-11-02|
|         442737|          3949|NO APLICA|  219.00|       2023-11-01|               null|759-476-096-508|           2023-11-02|
|         802156|           155|NO APLICA|   63.50|       2023-11-01|               null|200-927-832-435

In [21]:
#Contar numero de filas
print(df_clip_no_pago.count())
print(df_santander_no_pago.count())

16218
58


#Casting

In [22]:
df_clip_pago = df_clip_pago.withColumn("MONTO_D", col("MONTO").cast("double"))
df_santander_pago = df_santander_pago.withColumn("MONTO_D", col("MONTO").cast("double"))

In [23]:
#verificar tipos de datos
df_clip_pago.printSchema()
df_santander_pago.printSchema()

root
 |-- INICIO6_TARJETA: string (nullable = true)
 |-- FINAL4_TARJETA: string (nullable = true)
 |-- TIPO_TRX: string (nullable = true)
 |-- MONTO: string (nullable = true)
 |-- FECHA_TRANSACCION: string (nullable = true)
 |-- CODIGO_AUTORIZACION: string (nullable = true)
 |-- ID_BANCO: string (nullable = true)
 |-- FECHA_RECEPCION_BANCO: string (nullable = true)
 |-- MONTO_D: double (nullable = true)

root
 |-- BIN_6: string (nullable = true)
 |-- LAST4: string (nullable = true)
 |-- TIPO_TRX: string (nullable = true)
 |-- MONTO: double (nullable = true)
 |-- FECHA_TRANSACCION: date (nullable = true)
 |-- CODIGO_AUTORIZACION: string (nullable = true)
 |-- ID_BANCO: string (nullable = true)
 |-- FECHA_RECEPCION_BANCO: string (nullable = true)
 |-- MONTO_D: double (nullable = true)



#Conciliación

In [25]:
cond = [
    col("c.ID_BANCO")             == col("s.ID_BANCO"),         # 1) mismo ID
    col("c.INICIO6_TARJETA")      == col("s.BIN_6"),                   # 2) mismos 6 dígitos
    col("c.FINAL4_TARJETA")       == col("s.LAST4"),                # 3) mismos 4 últimos dígitos
    abs(col("c.MONTO_D") - col("s.MONTO_D")) <= 0.99,               # 4) diff monto ≤ 0.99
    to_date(col("c.FECHA_TRANSACCION")) ==
      to_date(col("s.FECHA_TRANSACCION"))                         # 5) misma fecha
]

matched = (
    df_clip_pago.alias("c")
    .join(df_santander_pago.alias("s"), cond, how="inner")
)

# Transacciones en Clip sin pareja en Santander
only_in_clip = (
    df_clip_pago.alias("c")
    .join(df_santander_pago.alias("s"), cond, how="left_anti")
)

# Transacciones en Santander sin pareja en Clip
only_in_santander = (
    df_santander_pago.alias("s")
    .join(df_clip_pago.alias("c"), cond, how="left_anti")
)

In [29]:
# Conciliadas
print("→ Conciliadas (match):")
matched.select(
    "c.ID_BANCO","c.INICIO6_TARJETA","c.FINAL4_TARJETA","c.MONTO_D","c.FECHA_TRANSACCION",
    "s.MONTO_D", "c.TIPO_TRX", "s.ID_BANCO", "s.FECHA_RECEPCION_BANCO", "s.CODIGO_AUTORIZACION"
).show(10,truncate=False)

# Sólo en Clip
print("→ Sólo en Clip (sin match):")
only_in_clip.show(10, truncate=False)  # muestra 10 filas

# Sólo en Santander
print("→ Sólo en Santander (sin match):")
only_in_santander.show(10, truncate=False)

→ Conciliadas (match):
+-------------+---------------+--------------+-------+-----------------+-------+--------+-------------+---------------------+-------------------+
|ID_BANCO     |INICIO6_TARJETA|FINAL4_TARJETA|MONTO_D|FECHA_TRANSACCION|MONTO_D|TIPO_TRX|ID_BANCO     |FECHA_RECEPCION_BANCO|CODIGO_AUTORIZACION|
+-------------+---------------+--------------+-------+-----------------+-------+--------+-------------+---------------------+-------------------+
|1-006-920-138|975488         |8840          |580.0  |2023-11-01       |580.0  |PAGADA  |1-006-920-138|2023-03-11           |1178               |
|1-068-687-103|449289         |4444          |1000.0 |2023-11-01       |1000.0 |PAGADA  |1-068-687-103|2023-03-11           |1753               |
|1-111-401-104|975199         |7422          |679.0  |2023-11-01       |679.0  |PAGADA  |1-111-401-104|2023-03-11           |629793             |
|1-210-718-304|442753         |6747          |135.5  |2023-11-01       |135.5  |PAGADA  |1-210-718-30

In [30]:
#Conteo de filas match
print(matched.count())

86771


In [31]:
#Conteo de filas solo en clip
print(only_in_clip.count())


60557


In [32]:
#Conteo de registros solo en santander
print(only_in_santander.count())

45567


#Conclusiones




Al realizar el tratamiento inicial de los datos nulos, duplicados y la filtración específica de las transacciones clasificadas como "PAGO" en los DataFrames de Clip y Santander, se observó un cambio importante en el número total de registros disponibles.

Inicialmente, los registros totales eran de **163,549** en Clip y 132,398 en Santander. Después del tratamiento, los registros se ajustaron ligeramente a **163,537** en Clip y **132,396** en Santander. Esta pequeña variación refleja que no existían registros con valores realmente nulos, mientras que los duplicados encontrados y eliminados fueron mínimos. Sin embargo, es relevante destacar que se identificaron registros en Clip que contenían la cadena de texto "null". Este hallazgo merece atención especial, pues indica que hay registros almacenados incorrectamente como strings, lo que podría causar confusión o errores en análisis futuros. Por lo tanto, es recomendable realizar un análisis detallado para identificar el origen de estos registros y evaluar su impacto en procesos posteriores.

En cuanto a las transacciones específicamente identificadas como pagos, se obtuvo un universo más reducido: **147,328** registros en Clip y **132,338** en Santander. Por otro lado, es notable mencionar que se identificaron **16,209** registros que no corresponden a pagos en Clip y apenas **58** en Santander, lo que resalta la relevancia de la clasificación precisa de los tipos de transacción.


Además, antes de realizar la conciliación final, se ejecutó un proceso adicional para transformar los montos almacenados originalmente como cadenas de texto a un formato numérico (double), facilitando así una comparación precisa de valores monetarios. Esto permitió verificar efectivamente si los montos transaccionales coinciden o presentan una diferencia aceptable, establecida dentro del rango de más o menos 0.99 pesos.

Finalmente, el proceso de conciliación reveló que **86,771** registros cumplían todas las condiciones establecidas: mismo ID, coincidencia exacta en los primeros 6 y últimos 4 dígitos de la tarjeta, fechas idénticas de transacción y montos que difieren como máximo en 0.99 pesos. Además, se detectaron registros únicos sin correspondencia entre ambas bases: **60,557** transacciones únicas de pagos en Clip y **45,567** en Santander.

| **Concepto**                              | **Clip**   | **Santander** |
| ----------------------------------------- | ---------- | ------------- |
| Registros iniciales totales               | 163,549    | 132,398       |
| Registros después de tratamiento          | 163,537    | 132,396       |
| Transacciones clasificadas como "PAGO"    | 147,328    | 132,338       |
| Transacciones clasificadas como "NO PAGO" | 16,216     | 58            |
| Transacciones conciliadas (match)         | **86,771** | **86,771**    |
| Transacciones únicas (sin conciliación)   | 60,557     | 45,567        |
